# Welcome to Jupyter!

Data
1.District of Ankara are obtained from https://en.wikipedia.org/wiki/Ankara_Province
2.Latitude and Longitude values are obtained by using "geocoder".
3.All data related to locations will be obtaine by using FourSquare API and Python Libraries.

In [1]:
!pip install BeautifulSoup4
!pip install geocoder
import pandas as pd
from bs4 import BeautifulSoup
!conda install -c conda-forge geopy --yes
import geocoder
import requests

     |████████████████████████████████| 115 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 3.3 MB/s eta 0:00:011
     |████████████████████████████████| 829 kB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 1.5 MB/s  eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=240b6c7f39dc4f0dd8bb45e2da16a384af6634dc0c7b6d4cdd1958e7654094f7
  Stored in directory: /home/jovyan/.cache/pip/wheels/6e/9c/ed/4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
Successfully built future
Solving environment: done

## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge


In [2]:

wiki_link = 'https://en.wikipedia.org/wiki/Ankara_Province'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0'}
wiki_page = requests.get(wiki_link, headers = headers)
wiki_page

<Response [200]>

In [3]:
soup = BeautifulSoup(wiki_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [4]:
rows = table.find_all('tr')

In [5]:
columns = [i.text.replace('\n', '') for i in rows[0].find_all('th')]
columns

['District', 'Population (2017)', 'Area (km²)', 'Density (per km²)']

In [6]:
df_ankara = pd.DataFrame(columns = columns)

In [7]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df_ankara = df_ankara.append(pd.Series(values, index = columns), ignore_index = True)

        df_ankara

In [8]:
df_ankara.head()

,District,Population (2017),Area (km²),Density (per km²)
0,Akyurt,32.863,369,89
1,Altındağ,371.366,123,3.019
2,Ayaş,12.289,1.041,12
3,Bala,21.682,1.851,12
4,Beypazarı,48.476,1.697,29


In [9]:
def get_latlng(arcgis_geocoder):
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Ankara, Turkey'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
districts = df_ankara['District']    
coordinates = [get_latlng(districts) for districts in districts.tolist()]

In [11]:
df_ankara_loc = df_ankara

df_ankara_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_ankara_loc['Latitude'] = df_ankara_coordinates['Latitude']
df_ankara_loc['Longitude'] = df_ankara_coordinates['Longitude']

In [12]:
df_ankara_loc.drop(columns="Population (2017)", axis=1, inplace=True)

In [13]:
df_ankara_loc.drop(columns="Density (per km²)", axis=1, inplace=True)

In [14]:
df_ankara_loc.drop(columns="Area (km²)", axis=1, inplace=True)

In [15]:
df_ankara_loc.head()

,District,Latitude,Longitude
0,Akyurt,40.13082,33.08719
1,Altındağ,39.94171,32.85445
2,Ayaş,40.01516,32.33270
3,Bala,39.55391,33.12352
4,Beypazarı,40.16811,31.92052


In [17]:
!pip install folium
import numpy as np
import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print("Libraries imported")

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  branca             conda-forge/noarch::branca-0.4.2-pyhd8ed1ab_0
  folium    

In [18]:
from geopy.geocoders import Nominatim 

address = "Çankaya, Ankara"

geolocator = Nominatim(user_agent = "Ankara_explorer")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinates of Ankara are {}, {}.".format(latitude, longitude))

The geographical coordinates of Ankara are 39.9207486, 32.8540093.


In [19]:
map_ankara = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(df_ankara_loc["Latitude"], df_ankara_loc["Longitude"], df_ankara_loc["District"]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=25,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_ankara)  
    
map_ankara

In [20]:
CLIENT_ID = "IMRIH02A42G4OOJEWRXGXOMWD3QXWQA3VN3PVDSMNVBGAR2X"
CLIENT_SECRET = "SBRBHICP1B5WHMZ3HVLHEQM04JZBV1FBTRMXZE0RKSJLOTQN" 
VERSION = "20212002"

In [21]:
df_ankara_loc.loc [0, "District"]
df_ankara_loc.loc [0, "District"]

'Akyurt'

In [22]:
neighborhood_latitude = df_ankara_loc.loc[0, "Latitude"]
neighborhood_longitude = df_ankara_loc.loc[0, "Longitude"] 

neighborhood_name = df_ankara_loc.loc[0, "District"] 

print("Latitude and longitude values of the neighborhood {} are {}, {}.".format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of the neighborhood Akyurt are 40.13082000000003, 33.08719000000008.


In [23]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IMRIH02A42G4OOJEWRXGXOMWD3QXWQA3VN3PVDSMNVBGAR2X&client_secret=SBRBHICP1B5WHMZ3HVLHEQM04JZBV1FBTRMXZE0RKSJLOTQN&v=20212002&ll=40.13082000000003,33.08719000000008&radius=500&limit=100'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60311180483975334d2ebfbc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Akyurt',
  'headerFullLocation': 'Akyurt, Ankara',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 18,
  'suggestedBounds': {'ne': {'lat': 40.13532000450003,
    'lng': 33.093064640939495},
   'sw': {'lat': 40.126319995500026, 'lng': 33.08131535906066}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5049c781e4b0a7a32d840eb1',
       'name': 'Meşhur Köfteci Bodur',
       'location': {'address': 'Cumhuriyet Meydani',
        'crossStreet': 'Cumhuriyet Meydani',
        'lat': 40.13071902888477,
        'lng': 33.08677622815274,
        'labeledLatLngs':

In [25]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Meşhur Köfteci Bodur,Turkish Restaurant,40.130719,33.086776
1,Akyurt kültür parkı,Park,40.133123,33.085071
2,Yurtalan Restaurant,Kebab Restaurant,40.132672,33.084306
3,Onur pide ve kebap salonu,Kebab Restaurant,40.130807,33.086422
4,Beyazıt Sofrası,Steakhouse,40.130542,33.081901
5,Yurtalan,Turkish Restaurant,40.132743,33.084385
6,Yurtalan Restaurant,Doner Restaurant,40.132641,33.084492
7,Akyurt Sofrası,Turkish Restaurant,40.130854,33.086709
8,Akyurt,Farm,40.131012,33.083330
9,Benliler Supermarket,Convenience Store,40.130252,33.081423


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

18 venues were returned by Foursquare.


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
ankara_venues = getNearbyVenues(names=df_ankara_loc['District'],
                                   latitudes=df_ankara_loc['Latitude'],
                                   longitudes=df_ankara_loc['Longitude']
                                  )

Akyurt
Altındağ
Ayaş
Bala
Beypazarı
Çamlıdere
Çankaya
Çubuk
Elmadağ
Etimesgut
Evren
Gölbaşı
Güdül
Haymana
Kahramankazan
Kalecik
Keçiören
Kızılcahamam
Mamak
Nallıhan
Polatlı
Pursaklar
Sincan
Şereflikoçhisar
Yenimahalle
Urban (9 districts)Altındağ, Çankaya, Etimesgut, Gölbaşı, Keçiören, Mamak, Pursaklar, Sincan, Yenimahalle
TOTAL


In [30]:
print(ankara_venues.shape)
ankara_venues.head(250)

(1681, 7)


,District,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akyurt,40.13082,33.08719,Meşhur Köfteci Bodur,40.130719,33.086776,Turkish Restaurant
1,Akyurt,40.13082,33.08719,Akyurt kültür parkı,40.133123,33.085071,Park
2,Akyurt,40.13082,33.08719,Yurtalan Restaurant,40.132672,33.084306,Kebab Restaurant
3,Akyurt,40.13082,33.08719,Beyazıt Sofrası,40.130542,33.081901,Steakhouse
4,Akyurt,40.13082,33.08719,Onur pide ve kebap salonu,40.130807,33.086422,Kebab Restaurant
...,...,...,...,...,...,...,...
245,Beypazarı,40.16811,31.92052,Sağlık Meslek Spor Salonu,40.173256,31.933203,Trail
246,Beypazarı,40.16811,31.92052,Tolunay Özaka Sağlık Meslek Lisesi,40.173212,31.933535,Volleyball Court
247,Beypazarı,40.16811,31.92052,Azra Market,40.155687,31.917767,Convenience Store
248,Beypazarı,40.16811,31.92052,akyazı,40.172203,31.936853,Residential Building (Apartment / Condo)


In [31]:
ankara_venues.groupby("District").count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Akyurt,29,29,29,29,29,29
Altındağ,100,100,100,100,100,100
Ayaş,39,39,39,39,39,39
Bala,12,12,12,12,12,12
Beypazarı,70,70,70,70,70,70
Elmadağ,51,51,51,51,51,51
Etimesgut,100,100,100,100,100,100
Evren,4,4,4,4,4,4
Gölbaşı,100,100,100,100,100,100


In [32]:
print('There are {} unique venue categories.'.format(len(ankara_venues['Venue Category'].unique())))

There are 225 unique venue categories.


In [33]:
ankara_onehot = pd.get_dummies(ankara_venues[['Venue Category']], prefix="", prefix_sep="")

ankara_onehot['Neighborhood'] = ankara_venues['District'] 

fixed_columns = [ankara_onehot.columns[-1]] + list(ankara_onehot.columns[:-1])
ankara_onehot = ankara_onehot[fixed_columns]

ankara_onehot

,Neighborhood,ATM,Advertising Agency,Airport Lounge,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Turkish Home Cooking Restaurant,Turkish Restaurant,Used Bookstore,Video Game Store,Vineyard,Volleyball Court,Water Park,Waterfront,Wedding Hall,Women's Store
0,Akyurt,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1677,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1679,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
ankara_onehot.shape

(1681, 226)

In [35]:
ankara_grouped = ankara_onehot.groupby('Neighborhood').mean().reset_index()
ankara_grouped

,Neighborhood,ATM,Advertising Agency,Airport Lounge,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Turkish Home Cooking Restaurant,Turkish Restaurant,Used Bookstore,Video Game Store,Vineyard,Volleyball Court,Water Park,Waterfront,Wedding Hall,Women's Store
0,Akyurt,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.103448,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000
1,Altındağ,0.000000,0.00,0.000000,0.02,0.000000,0.02,0.020000,0.000000,0.000000,...,0.000000,0.050000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000
2,Ayaş,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.025641,...,0.000000,0.128205,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000
3,Bala,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.166667,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000
4,Beypazarı,0.000000,0.00,0.000000,0.00,0.014286,0.00,0.000000,0.028571,0.000000,...,0.014286,0.100000,0.00,0.000000,0.0000,0.014286,0.000000,0.00,0.000000,0.000000
5,Elmadağ,0.019608,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.019608,0.019608,...,0.000000,0.019608,0.00,0.000000,0.0000,0.000000,0.019608,0.00,0.000000,0.000000
6,Etimesgut,0.000000,0.00,0.000000,0.00,0.040000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.090000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000
7,Evren,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.25,0.000000,0.000000
8,Gölbaşı,0.000000,0.00,0.000000,0.00,0.030000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.100000,0.00,0.000000,0.0000,0.000000,0.010000,0.00,0.000000,0.000000
9,Güdül,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.125000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000


In [36]:
ankara_grouped.shape

(27, 226)

In [37]:
num_top_venues = 10

for hood in ankara_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ankara_grouped[ankara_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Akyurt----
                venue  freq
0                Farm  0.14
1  Turkish Restaurant  0.10
2    Doner Restaurant  0.07
3                Park  0.07
4    Kebab Restaurant  0.07
5            Mountain  0.03
6       Big Box Store  0.03
7          Steakhouse  0.03
8             Stadium  0.03
9    Botanical Garden  0.03


----Altındağ----
                venue  freq
0                Café  0.09
1      History Museum  0.08
2       Jewelry Store  0.05
3  Turkish Restaurant  0.05
4       Historic Site  0.05
5               Hotel  0.04
6             Theater  0.04
7    Kebab Restaurant  0.03
8          Steakhouse  0.03
9         Snack Place  0.02


----Ayaş----
                     venue  freq
0       Turkish Restaurant  0.13
1                     Park  0.08
2               Restaurant  0.05
3                     Lake  0.05
4               Food Court  0.05
5                   Buffet  0.05
6              Coffee Shop  0.03
7               Steakhouse  0.03
8  Comfort Food Restaurant  0.03
9    

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
ankara_venues_sorted = pd.DataFrame(columns=columns)
ankara_venues_sorted['Neighborhood'] = ankara_grouped['Neighborhood']

for ind in np.arange(ankara_grouped.shape[0]):
    ankara_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ankara_grouped.iloc[ind, :], num_top_venues)

ankara_venues_sorted.head(27)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Farm,Turkish Restaurant,Park,Doner Restaurant,Kebab Restaurant,Basketball Court,Garden,Stadium,Café,Steakhouse
1,Altındağ,Café,History Museum,Turkish Restaurant,Jewelry Store,Historic Site,Hotel,Theater,Steakhouse,Kebab Restaurant,Art Museum
2,Ayaş,Turkish Restaurant,Park,Restaurant,Lake,Food Court,Buffet,Pool,Taxi Stand,Grocery Store,Farm
3,Bala,Electronics Store,Turkish Restaurant,Big Box Store,Garden,Park,Café,Mountain,Steakhouse,Bakery,Convenience Store
4,Beypazarı,Turkish Restaurant,History Museum,Hotel,Scenic Lookout,Bakery,Motel,Tea Room,Restaurant,Bed & Breakfast,Historic Site
5,Elmadağ,Café,Park,Campground,Kebab Restaurant,Convenience Store,Dessert Shop,Bar,Kofte Place,Food Truck,Italian Restaurant
6,Etimesgut,Turkish Restaurant,Café,Restaurant,Arcade,Gym,Steakhouse,Shopping Mall,Snack Place,Kebab Restaurant,Park
7,Evren,River,Waterfront,Grocery Store,Park,Women's Store,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
8,Gölbaşı,Café,Turkish Restaurant,Park,Breakfast Spot,Restaurant,Coffee Shop,Dessert Shop,Arcade,Middle Eastern Restaurant,Bakery
9,Güdül,Pharmacy,Castle,Soccer Stadium,Lake,Department Store,Park,Turkish Restaurant,Botanical Garden,Falafel Restaurant,Doner Restaurant


In [49]:
!pip install KMeans
from sklearn.cluster import KMeans as kmeans

In [50]:
ankara_merged = df_ankara

In [51]:
import numpy as np
kmeans = KMeans()
X = np.random.rand(27, 2)
kmeans.fit(X)
ankara_merged['Cluster Labels'] = kmeans.labels_

ankara_merged = ankara_merged.join(ankara_venues_sorted.set_index('Neighborhood'), on='District')
ankara_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Farm,Turkish Restaurant,Park,Doner Restaurant,Kebab Restaurant,Basketball Court,Garden,Stadium,Café,Steakhouse
1,Altındağ,Café,History Museum,Turkish Restaurant,Jewelry Store,Historic Site,Hotel,Theater,Steakhouse,Kebab Restaurant,Art Museum
2,Ayaş,Turkish Restaurant,Park,Restaurant,Lake,Food Court,Buffet,Pool,Taxi Stand,Grocery Store,Farm
3,Bala,Electronics Store,Turkish Restaurant,Big Box Store,Garden,Park,Café,Mountain,Steakhouse,Bakery,Convenience Store
4,Beypazarı,Turkish Restaurant,History Museum,Hotel,Scenic Lookout,Bakery,Motel,Tea Room,Restaurant,Bed & Breakfast,Historic Site
5,Elmadağ,Café,Park,Campground,Kebab Restaurant,Convenience Store,Dessert Shop,Bar,Kofte Place,Food Truck,Italian Restaurant
6,Etimesgut,Turkish Restaurant,Café,Restaurant,Arcade,Gym,Steakhouse,Shopping Mall,Snack Place,Kebab Restaurant,Park
7,Evren,River,Waterfront,Grocery Store,Park,Women's Store,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
8,Gölbaşı,Café,Turkish Restaurant,Park,Breakfast Spot,Restaurant,Coffee Shop,Dessert Shop,Arcade,Middle Eastern Restaurant,Bakery
9,Güdül,Pharmacy,Castle,Soccer Stadium,Lake,Department Store,Park,Turkish Restaurant,Botanical Garden,Falafel Restaurant,Doner Restaurant


In [53]:
ks = 3

ankara_grouped_clustering = ankara_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=ks, random_state=0).fit(ankara_grouped_clustering)

kmeans.labels_[1:10]

array([2, 2, 2, 2, 2, 2, 1, 2, 2], dtype=int32)

In [45]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(ks)
ys = [i+x+(i*x)**2 for i in range(ks)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ankara_merged['Latitude'], ankara_merged['Longitude'], ankara_merged['District'], ankara_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

IndexError: list index out of range

In [54]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 0, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Haymana,Hotel,Restaurant,Café,Diner,Water Park,Hot Spring,Bathing Area,Supermarket,Bath House,Recreation Center
15,Kalecik,Nightclub,Smoke Shop,Diner,Buffet,Canal,Cafeteria,Turkish Coffeehouse,Park,Turkish Restaurant,Furniture / Home Store
20,Polatlı,Café,Restaurant,Park,Turkish Restaurant,Convenience Store,Kebab Restaurant,Soup Place,Furniture / Home Store,Pharmacy,Supermarket


In [55]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 1, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Çubuk,Café,Turkish Restaurant,Park,Bistro,Restaurant,Mobile Phone Shop,Kebab Restaurant,Convenience Store,Gym / Fitness Center,Gaming Cafe
11,Gölbaşı,Café,Turkish Restaurant,Park,Breakfast Spot,Restaurant,Coffee Shop,Dessert Shop,Arcade,Middle Eastern Restaurant,Bakery
17,Kızılcahamam,Hotel,Turkish Restaurant,Steakhouse,Café,Restaurant,Pool,Park,Hot Spring,Kofte Place,Bus Station
19,Nallıhan,Turkish Restaurant,Café,Mountain,Seafood Restaurant,Bistro,Pizza Place,Auto Garage,Diner,Fast Food Restaurant,Electronics Store
24,Yenimahalle,Café,Seafood Restaurant,Bakery,Turkish Restaurant,Arcade,Gym / Fitness Center,Sporting Goods Shop,Restaurant,Kokoreç Restaurant,Pub


In [56]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 2, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Farm,Turkish Restaurant,Park,Doner Restaurant,Kebab Restaurant,Basketball Court,Garden,Stadium,Café,Steakhouse
21,Pursaklar,Café,Park,Bakery,Gym / Fitness Center,Arcade,Bagel Shop,Mobile Phone Shop,Clothing Store,Turkish Restaurant,Plaza
26,TOTAL,Soup Place,Fast Food Restaurant,Park,Pastry Shop,Dive Bar,Food Truck,Food Court,Disc Golf,Kofte Place,Kokoreç Restaurant


In [57]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 3, ankara_merged.columns[[0] + list(range(5, ankara_merged.shape[1]))]]

,District,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,"Urban (9 districts)Altındağ, Çankaya, Etimesgu...",Coffee Shop,Bookstore,Turkish Restaurant,Hotel,Sandwich Place,Pub,Gym,Art Gallery,Sporting Goods Shop
